### Perspective API Exploration

First, we have a dataset of Wikipedia comments made available by Jigsaw, a subsidiary of Google that created the Perspective tool. The dataset includes a unique comment id, the text of the comment, and a series of binary labels applied by human raters: "toxic," "severe_toxic," "obscene," "threat," "insult," and "identity_hate." I have appended the "score" column, which represents the toxicity score assigned to the comment text by the live version of the Perspective API. The data is available under a CC0 license.

## Step 1

In [1]:
import pandas as pd
import time

df = pd.read_csv('labeled_and_scored_comments.csv')

In [2]:
df.sort_values(['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate
1158,61a85266de70cca0,Можешь говорить по русски.,0.000004,0,0,0,0,0,0
5246,030a9ba0b32b4456,Rodzina Krusenstern Potomkowie Admirała Adama ...,0.000009,0,0,0,0,0,0
11040,a6814193ac6b4705,The opening paragraph states that the game is ...,0.000091,0,0,0,0,0,0
24192,f0d6dee96e057a5b,The book is used as a source in a number of ar...,0.000116,0,0,0,0,0,0
2332,cbb9268d3c7429c1,Vals Vienés \n |Rumba \n |Cha Cha Chá(ciclo 7),0.000131,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
19994,1429a2c930688317,you fucking piece of shit,0.997278,1,1,1,0,1,0
28728,d82e45eb103ad07b,fuck off you stupid aspy asshole,0.997736,1,1,1,0,1,0
25535,e9b92c6c6b01aeef,FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIE...,0.997982,1,1,1,0,1,0
25945,e783fd267f3a9d3b,FUCK WIKIPEDIA ON WHEELS! \n\nFuck off wikiped...,0.998136,1,1,1,0,1,0


I've also included a function to make calls to the Perspective API for your own testing. You will need to generate your own API key according to the instructions in the assignment.

In [3]:
from googleapiclient.discovery import build
import json

def get_toxicity_score(comment):
    
  API_KEY = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX' # Put your API key here
    
  client = build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
  )

  analyze_request = {
  'comment': { 'text': comment },
  'requestedAttributes': {'TOXICITY': {}}
  }
    
  response = client.comments().analyze(body=analyze_request).execute()
  toxicity_score = response["attributeScores"]["TOXICITY"]["summaryScore"]["value"]
    
  return toxicity_score

We can call this function with original comments like this:

In [4]:
get_toxicity_score("I love you")

0.040628307

Due to the constraint of one request per second, we should insert a time.sleep() statement between consecutive API calls, which will pause execution for one second.

In [5]:
comment_list = ['hello','how are you','fine thanks','what',"shut up","screw you","Obama","Trump","Biden","huge","IDIOT","idiot","curse","politics","audacity"]

score_list=[]
for comment in comment_list:
    score = get_toxicity_score(comment)
    score_list.append(score)
    print(comment, score)
    time.sleep(1)

hello 0.02397547
how are you 0.018187003
fine thanks 0.014022041
what 0.024833864
shut up 0.7382047
screw you 0.5958905
Obama 0.033074442
Trump 0.04131502
Biden 0.023288755
huge 0.04234509
IDIOT 0.8831064
idiot 0.8086813
curse 0.29070088
politics 0.020824814
audacity 0.038911518


In [6]:
sum(score_list)

3.5973617970000005

In [7]:
new_df = pd.DataFrame({'comment': comment_list, 'score': score_list})

In [8]:
new_df.describe()

,score
count,15.000000
mean,0.239824
std,0.334285
min,0.014022
25%,0.023632
50%,0.038912
75%,0.443296
max,0.883106


In [9]:
new_df

,comment,score
0,hello,0.023975
1,how are you,0.018187
2,fine thanks,0.014022
3,what,0.024834
4,shut up,0.738205
5,screw you,0.595890
6,Obama,0.033074
7,Trump,0.041315
8,Biden,0.023289
9,huge,0.042345


In [10]:
df.head()

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0001ea8717f6de06,Thank you for understanding. I think very high...,0.075638,0,0,0,0,0,0
1,000247e83dcc1211,:Dear god this site is horrible.,0.450459,0,0,0,0,0,0
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0.667964,0,0,0,0,0,0
3,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0.068434,0,0,0,0,0,0
4,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0.151724,0,0,0,0,0,0


In [11]:
df[(df['toxic']==1)|(df['severe_toxic']==1)]

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate
8,00091c35fa9d0465,"== Arabs are committing genocide in Iraq, but ...",0.635929,1,0,0,0,0,0
24,0013fed3aeae76b7,DJ Robinson is gay as hell! he sucks his dick ...,0.989706,1,0,1,0,1,1
29,0017d4d47894af05,":Fuck off, you anti-semitic cunt. |",0.959459,1,0,1,0,1,0
37,001d739c97bc2ae4,How dare you vandalize that page about the HMS...,0.564152,1,0,0,0,0,0
38,001eff4007dbb65b,"::No, he is an arrogant, self serving, immatur...",0.956893,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...
41297,94a6beeb87ab68c0,"Damn, those are some rancid sources.",0.821990,1,0,1,0,0,0
41299,94a5024323152cd1,"==Why does it bother you, fuckface?89.123.100....",0.989706,1,0,1,0,1,0
41325,948cb4052152869b,WHAT? That's past stuff. Stop being an insti...,0.515460,1,0,0,0,0,0
41332,9481cd7393b583c9,"RE: \n\nIt's a fucking album cover, how the fu...",0.932649,1,0,1,0,0,0


In [12]:
threshold = 0.5

df['prediction'] = (df['score'] > threshold).astype(int)
df['prediction'].value_counts()

0    34925
1     6413
Name: prediction, dtype: int64

In [13]:
from sklearn.metrics import confusion_matrix

In [14]:
confusion_matrix(df['toxic'], df['prediction'])

array([[34677,  2718],
       [  248,  3695]], dtype=int64)

In [15]:
pred_toxic=df[(df['toxic']==1)|(df['prediction']==1)]
pred_toxic

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,prediction
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0.667964,0,0,0,0,0,0,1
8,00091c35fa9d0465,"== Arabs are committing genocide in Iraq, but ...",0.635929,1,0,0,0,0,0,1
9,000968ce11f5ee34,Please stop. If you continue to vandalize Wiki...,0.604778,0,0,0,0,0,0,1
12,000bafe2080bba82,. \n\n Jews are not a race because y...,0.628810,0,0,0,0,0,0,1
17,001068b809feee6b,""" \n\n ==balance== \n This page has one senten...",0.729625,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
41317,9495e55ccf912a2a,"""\n\n """"Endocinal Jubachina System"""" \n\nThis ...",0.647981,0,0,0,0,0,0,1
41325,948cb4052152869b,WHAT? That's past stuff. Stop being an insti...,0.515460,1,0,0,0,0,0,1
41330,948628822e4dd6c7,"""\n\nThese sources don't exactly exude a sense...",0.559237,0,0,0,0,0,0,1
41332,9481cd7393b583c9,"RE: \n\nIt's a fucking album cover, how the fu...",0.932649,1,0,1,0,0,0,1


#### Test Hypothesis: In the content marked as severely toxic and obscene by the algorithm, the threat and identity hate often do not intervene (one is recognized while the other is not). Such content will include a profanity but not a verb or a verb but not a profanity.

In [16]:
pred_toxic_severe=pred_toxic[(pred_toxic['severe_toxic']==1)]

In [17]:
pred_toxic_severe_obscene=pred_toxic_severe[(pred_toxic_severe['obscene']==1)]
pred_toxic_severe_obscene

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,prediction
330,1a3ffaa9e319104c,==Yo mama== \n This is yo mama and I know that...,0.959383,1,1,1,0,1,0,1
332,1a44cad92a0c1c96,==Son of a bitch== \n\n Hey you fucking neo-Na...,0.992705,1,1,1,0,1,1,1
338,1a4c6302cea3ffe2,Dick Cheney will fucking kill all of those god...,0.951699,1,1,1,1,1,1,1
430,20cdeb8ee999a478,== You know what smart arrse........phuck a bi...,0.928721,1,1,1,0,1,0,1
447,2412ef2ce6f87a73,"SSJ, the only way that fucking pitiful excuse ...",0.977829,1,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...
41017,9622c82f97eb26cd,She is a Motherfucker \n\nI will block you Mot...,0.967966,1,1,1,1,1,1,1
41053,95eda78799f2f83c,i forgot to sign \nReed210 \n\nPS Does your di...,0.975341,1,1,1,0,1,0,1
41071,95ca65d4cd31d3dd,HOPE YOUR HEAD GETS CUT OFF AND SOMEONE WIPES ...,0.980211,1,1,1,1,1,0,1
41181,95390a4e0cd90ad3,"Fuck You Asshole \n\nFuck You, you cum-guzzlin...",0.991540,1,1,1,0,1,0,1


In [18]:
from collections import Counter
Counter(" ".join(pred_toxic_severe_obscene["comment_text"]).split()).most_common(100)

[('you', 1145),
 ('fuck', 923),
 ('FUCK', 914),
 ('YOU', 905),
 ('a', 728),
 ('your', 642),
 ('fucksex', 624),
 ('MY', 432),
 ('FAGGOT', 420),
 ('HUGE', 418),
 ('I', 373),
 ('YOUR', 372),
 ('DOG', 354),
 ('SUCK', 327),
 ('COCK', 317),
 ('D', 317),
 ('dickhead', 313),
 ('ALL!!', 312),
 ('and', 308),
 ('You', 306),
 ('the', 305),
 ('Go', 283),
 ('yourself!', 265),
 ('stupid', 255),
 ('of', 243),
 ('sucks', 243),
 ('nigger', 242),
 ('A', 236),
 ('my', 222),
 ('veggietales', 212),
 ('ARE', 198),
 ('fucking', 194),
 ('to', 191),
 ('FUCKER', 186),
 ('MOTHJER', 185),
 ('COCKSUCKER!', 184),
 ('in', 173),
 ('pussy', 168),
 ('is', 164),
 ('i', 162),
 ('suck', 150),
 ('SHIT', 124),
 ('FUCKING', 122),
 ('ass', 118),
 ('YOU!', 115),
 ('bastard', 112),
 ('cock', 106),
 ("Pro-Assad.Hanibal911You're", 106),
 ('want', 102),
 ('itsuck', 101),
 ('F', 100),
 ('are', 99),
 ('UUUUUU', 96),
 ('CCCCCC', 96),
 ('KKKKKK', 96),
 ('=====', 96),
 ('UU', 96),
 ('C', 96),
 ('KK', 96),
 ('FFFFF', 96),
 ('Fucking', 95

In [19]:
threat=pred_toxic_severe_obscene[(pred_toxic_severe_obscene['threat']==1)&(pred_toxic_severe_obscene['identity_hate']==0)]
threat

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,prediction
447,2412ef2ce6f87a73,"SSJ, the only way that fucking pitiful excuse ...",0.977829,1,1,1,1,1,0,1
2593,eca0e12c9c0311cd,===Apology=== \n I would like to sincerely apo...,0.967279,1,1,1,1,1,0,1
2702,f77b3160efb88139,You are a useless cunt. You send me anymore me...,0.989706,1,1,1,1,1,0,1
4690,c3f15791ddceb838,"Oh, by the way, your mamma's a silly old fart....",0.968652,1,1,1,1,1,0,1
6190,f4dca9e513e0366f,"Hey \n\nhey faggot, are you dead yet? or are y...",0.974100,1,1,1,1,1,0,1
7411,e1416bf0b4bbfd24,I Want You \n\nI wanna rape you good. I want y...,0.959035,1,1,1,1,1,0,1
9312,c2a04ea79dcfd1a4,COME DUCT-TAKE YOU AND RAPE YOU TILL YOU DIE F...,0.967717,1,1,1,1,1,0,1
13167,83d6abb2cded7619,Fuck you cunt \n\n \nHow fucking dare u judge ...,0.984038,1,1,1,1,1,0,1
14180,72d35581d815b083,thanks for all your hard work \n\nlook here yo...,0.959105,1,1,1,1,1,0,1
17281,3f252c5b8db86046,Christmas Message \n\nI hope you die in hell y...,0.966643,1,1,1,1,1,0,1


In [20]:
identity=pred_toxic_severe_obscene[(pred_toxic_severe_obscene['threat']==0)&(pred_toxic_severe_obscene['identity_hate']==1)]
identity

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,prediction
332,1a44cad92a0c1c96,==Son of a bitch== \n\n Hey you fucking neo-Na...,0.992705,1,1,1,0,1,1,1
797,3f951ed6662d0020,OSUCK MY FUCKIN GAWDAM DICK YOU FAGGGOTSpinion,0.960409,1,1,1,0,1,1,1
836,4368d67cfeb54881,Deleted 3-2 \n You a bitch ass nigga for dele...,0.986299,1,1,1,0,1,1,1
1753,958e1f89dcb173ba,YOUR A STILL A GAY FAGGOT GO SUCK MY LEFT NUT ...,0.969081,1,1,1,0,1,1,1
1794,996dbd83299bde47,== slipknots are fags == \n\n slipknot sucks d...,0.979524,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...
38974,a11f0fe86a75d5d8,ZHANZHAO IS A MOTHERFUCKING PUSSY INDIAN-CHINE...,0.948132,1,1,1,0,1,1,1
39007,a0fda2f0f4148c92,GAY..... \n\nI THINK YOUR REALLY REALLY GAY AN...,0.932706,1,1,1,0,1,1,1
39633,9d8624f094ce679f,"STOP SUCKING THAT FATASS' DICK, YOU NIGGERLOVI...",0.972255,1,1,1,0,1,1,1
40423,9973cc45a10db3f5,"""\n\nhaha you are a dumb shit. what's so wrong...",0.892599,1,1,1,0,1,1,1


In [21]:
neither=pred_toxic_severe_obscene[(pred_toxic_severe_obscene['threat']==0)&(pred_toxic_severe_obscene['identity_hate']==0)]
neither

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,prediction
330,1a3ffaa9e319104c,==Yo mama== \n This is yo mama and I know that...,0.959383,1,1,1,0,1,0,1
430,20cdeb8ee999a478,== You know what smart arrse........phuck a bi...,0.928721,1,1,1,0,1,0,1
859,471cc216f314389c,== Baseball is for rednecks and alcoholics. ==...,0.948086,1,1,1,0,1,0,1
1027,565d151901ac2063,== Laura Bush Article == \n\n What the fuck di...,0.989706,1,1,1,0,1,0,1
2107,b505f2de5de26755,wikepedia suck greesy ass hiars and is a websi...,0.968543,1,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
40860,97003050132a4384,FUCK YOU DICK LICKS! \n\nYou guys will never b...,0.976342,1,1,1,0,0,0,1
40881,96d895add7fb5711,vandalising the page \n\nHow did i vandalise t...,0.973601,1,1,1,0,1,0,1
41053,95eda78799f2f83c,i forgot to sign \nReed210 \n\nPS Does your di...,0.975341,1,1,1,0,1,0,1
41181,95390a4e0cd90ad3,"Fuck You Asshole \n\nFuck You, you cum-guzzlin...",0.991540,1,1,1,0,1,0,1


In the new dataset called "neither", you can see the statements that were not classified as identity hate or threats. They're a lot more generic, simply expressing hatred but not towards something specific.

In [22]:
threshold = 0.8

df['prediction_obscene'] = (df['score'] > threshold).astype(int)
df['prediction_obscene'].value_counts()

0    38140
1     3198
Name: prediction_obscene, dtype: int64

In [23]:
df.drop(['id','prediction'],axis=1)

,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate,prediction_obscene
0,Thank you for understanding. I think very high...,0.075638,0,0,0,0,0,0,0
1,:Dear god this site is horrible.,0.450459,0,0,0,0,0,0,0
2,"""::: Somebody will invariably try to add Relig...",0.667964,0,0,0,0,0,0,0
3,""" \n\n It says it right there that it IS a typ...",0.068434,0,0,0,0,0,0,0
4,""" \n\n == Before adding a new product to the l...",0.151724,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
41333,Lesbian Teenager \nHow about the fact she was ...,0.797672,1,0,0,0,0,0,0
41334,"Crack, also called cocaine is very good for yo...",0.240977,0,0,0,0,0,0,0
41335,REDIRECT Talk:The Test (The O.C.),0.012671,0,0,0,0,0,0,0
41336,A question concerning your edit to my user-pag...,0.227474,0,0,0,0,0,0,0


Now, I would like to score how well the treshhold of 0.8 corresponds to the Obscene comments by calcluating the Root Means Square Error.

In [24]:
from sklearn import metrics
print("RMSE: %f" % (metrics.mean_squared_error(df['obscene'],df['prediction_obscene'], squared=True)))

RMSE: 0.034472


In [25]:
print("RMSE: %f" % (metrics.mean_squared_error(df['toxic'],df['prediction'], squared=True)))

RMSE: 0.071750


As you can see, the square error is relatively small - most values are above the threshold of 0.8 and 0.5 for the API-recognized Obscene and Toxic comments respectively.

In [26]:
print("RMSE: %f" % (metrics.mean_squared_error(df['severe_toxic'],df['prediction'], squared=True)))

RMSE: 0.146016


However, the squared error is larger when using the prediction of score>0.5 for the severely toxic comment. I expect the scores above 0.7 to fit that better.

In [27]:
threshold = 0.75

df['prediction_severe'] = (df['score'] > threshold).astype(int)
df['prediction_severe'].value_counts()

0    37655
1     3683
Name: prediction_severe, dtype: int64

In [28]:
print("RMSE: %f" % (metrics.mean_squared_error(df['severe_toxic'],df['prediction_severe'], squared=True)))

RMSE: 0.080120


The higher the score, the more likely it is to be marked as severely toxic.

In [29]:
threshold = 0.9

df['prediction_severe1'] = (df['score'] > threshold).astype(int)
df['prediction_severe1'].value_counts()

0    39381
1     1957
Name: prediction_severe1, dtype: int64

In [30]:
print("RMSE: %f" % (metrics.mean_squared_error(df['severe_toxic'],df['prediction_severe1'], squared=True)))

RMSE: 0.039141


## Note: I accidentaly ran a portion of the analysis planned on the csv file. Below is the API testing - this is what I believe counts for the assignment. However, I've decided to just keep the top part in the assignment too for comparison reasons (and because it took a while).

Lets return a toxicity score as determined by the API from the csv file.

In [31]:
score_list=[]
for i in threat['comment_text']:
    score = get_toxicity_score(i)
    score_list.append(score)
    print(i, score)
    time.sleep(1)

SSJ, the only way that fucking pitiful excuse for a site will ever get noticed is if you record yourself beating yourself in the head with your DS and then shoving it all the way down your throat until you choke to death on it. Burn in hell, geeky motherfucker! 0.9358428
===Apology=== 
 I would like to sincerely apologize to you being an asshole! Why would you report me to the administrators? Not man enough to challenge me motherfucker? Here's my email address karpgarr@msu.edu and if you wanna fucking call me do it here 847-579-1153. I WILL FUCKING RUN LAPS ON YOU DANIEL FAG LEVICK, YOU JEW. I want you to know that all the shit you talk will come back and fuck you up the ASS! I hope you die in your sleep! I will fuck up your life so bad you internet douchebag! I will do your mom and favorite aunt at the same time! I am gonna come over and eat all of your food and then take a shit and not flush! Don't fuck with the jesus man, ok jew, don't! Remember I may not be able to say this to your

In [32]:
print(score_list)

[0.9358428, 0.88651687, 0.97926265, 0.96751314, 0.95068264, 0.88651687, 0.7663013, 0.9863082, 0.9189384, 0.8742253, 0.96176314, 0.97926265, 0.9107452, 0.9863082, 0.9358428, 0.9583344]


In [33]:
score_list_iden=[]
for i in identity['comment_text'][0:43]:
    score = get_toxicity_score(i)
    score_list_iden.append(score)
    print(i, score)
    time.sleep(1)

==Son of a bitch== 

 Hey you fucking neo-Nazi, murkin son of a bitch. FUck you. Get the fuck out of this country or move to US of fucking A you motherfucker.  

 P.S> 

 Don't let the door hit your ass on the fucking way out, you fucking bushite. 0.9646382
OSUCK MY FUCKIN GAWDAM DICK YOU FAGGGOTSpinion 0.97926265
Deleted 3-2  
 You a bitch ass nigga for deleting my nigga 3-2s page you fuckin piece of shit. ever set urself in houston you will get capped. 0.95068264
YOUR A STILL A GAY FAGGOT GO SUCK MY LEFT NUT AND CUM ALL OVER MY PUSSSSSSSSSSY,asswipe 0.9448569
== slipknots are fags == 

 slipknot sucks dick 
  and suck at all times i thik slipknot is stupid and gay and there all girls 

  

  
 This goes to slip not shut the fuck up ur gay 0.93413806
== WHAT is going on here??? == 

 Aug 23, 10:22PM CST 
 This is the page text I am getting for this article: 

 Rip the sacred flesh Sodomize the holy asshole Drink the red blood of the mother of earth Masturbation on the dead body of chr

In [34]:
mix_threat = pd.concat([threat['comment_text'], threat['score']], axis=1)
mix_threat['API_score']=score_list
mix_threat=pd.DataFrame(mix_threat)

In [35]:
mix_identity = pd.concat([identity['comment_text'][0:43], identity['score'][0:43]], axis=1)
mix_identity['API_score']=score_list_iden
mix_identity=pd.DataFrame(mix_identity)

In [36]:
mix_threat

,comment_text,score,API_score
447,"SSJ, the only way that fucking pitiful excuse ...",0.977829,0.935843
2593,===Apology=== \n I would like to sincerely apo...,0.967279,0.886517
2702,You are a useless cunt. You send me anymore me...,0.989706,0.979263
4690,"Oh, by the way, your mamma's a silly old fart....",0.968652,0.967513
6190,"Hey \n\nhey faggot, are you dead yet? or are y...",0.974100,0.950683
7411,I Want You \n\nI wanna rape you good. I want y...,0.959035,0.886517
9312,COME DUCT-TAKE YOU AND RAPE YOU TILL YOU DIE F...,0.967717,0.766301
13167,Fuck you cunt \n\n \nHow fucking dare u judge ...,0.984038,0.986308
14180,thanks for all your hard work \n\nlook here yo...,0.959105,0.918938
17281,Christmas Message \n\nI hope you die in hell y...,0.966643,0.874225


In [37]:
mix_identity

,comment_text,score,API_score
332,==Son of a bitch== \n\n Hey you fucking neo-Na...,0.992705,0.964638
797,OSUCK MY FUCKIN GAWDAM DICK YOU FAGGGOTSpinion,0.960409,0.979263
836,Deleted 3-2 \n You a bitch ass nigga for dele...,0.986299,0.950683
1753,YOUR A STILL A GAY FAGGOT GO SUCK MY LEFT NUT ...,0.969081,0.944857
1794,== slipknots are fags == \n\n slipknot sucks d...,0.979524,0.934138
2558,"== WHAT is going on here??? == \n\n Aug 23, 10...",0.903912,0.655660
2569,fuckk u jayz is gayy,0.989706,0.839334
4130,Thompson is an asshole who should die like a d...,0.978773,0.964638
4197,super man was a crippled mo fo who had a bad t...,0.948712,0.918938
4402,"go to court? \n Shit, cause the judge look ju...",0.972633,0.896046


### Test Hypothesis:  In the content marked as severely toxic and obscene by the algorithm, the threat and identity hate often do not intervene (one is recognized while the other is not). Such content will include a profanity but not a verb or a verb but not a profanity. Additionally, I expect the RMSE between the table score and the API score for such explicit content to be under 0.05.

First, I sorted the csv file by labels to only include those comments that users marked as severely toxic and obscene. The main reason I did that was to work with the most severe cases, but severe in different ways. Therefore, I have chosen chosen the labels for content recognized as a (threat but not identity hate) and (identity hate but not threat). The threshold I used for toxic was 0.5, severely toxic was 0.75, and obscene was 0.8. Those tests were run on the csv data due to time constraints with the API.

Additionally, I have restricted the identity list due to the switch to another language that created an error for my API and my computer's inability to handle a block of Jupyter code for longer than a minute at a time. 

In [38]:
one=pd.DataFrame(Counter(" ".join(mix_identity["comment_text"]).split()).most_common(30))
one.columns = ['Identity Hate Word','Frequency']
two=pd.DataFrame(Counter(" ".join(mix_threat["comment_text"]).split()).most_common(30))
two.columns=['Threat Word','Frequency']
one

,Identity Hate Word,Frequency
0,FAGGOT,417
1,HUGE,417
2,You,251
3,stupid,244
4,nigger,242
5,the,51
6,you,47
7,of,43
8,a,42
9,and,32


In [39]:
two

,Threat Word,Frequency
0,you,38
1,your,35
2,and,23
3,I,19
4,to,17
5,the,14
6,YOU,9
7,me,8
8,You,8
9,fucking,7


As you can see in the lists of 20 most common words in both texts identified as threats and identity hate, the words usually perpetrate to different factors. In the case of identity, most of the words are slurs, degrading words, and express 'passinate' hatred. In the case of threat, the words refer to actions and verbs of profanity.

In [40]:
print("RMSE: %f" % (metrics.mean_squared_error(mix_threat['score'],mix_threat['API_score'], squared=True)))

RMSE: 0.004516


In [41]:
print("RMSE: %f" % (metrics.mean_squared_error(mix_identity['score'],mix_identity['API_score'], squared=True)))

RMSE: 0.003615


As you can see from the values above, my prediction on the RMSE was correct - the error is significantly smaller when squared. However, the regular RME would be slightly above 0.5 for both of those values. Some particular differences are visible in the identity dataset. The API frequently underestimated the toxicity of the comment by at least 5%.

#### Result analysis: 

As expected, most of the values provided by the API algorithm were close to the values in the dataset. It was interesting to see how certain words, particularly profanities, would have a significant influence on the toxicity score. I also noticed that capitalization of words lead to higher scores, perhaps this could be tested somehow through ASCII.